### IMPORTS ###

In [ ]:
import pandas as pd
import requests
from datetime import date
from datetime import datetime
import os
import time
from datetime import datetime, timezone, timedelta
import json

## FUNCTIONS ##


### DATA MANIPULATION FUNCTIONS ###

In [ ]:
# Creates columns on a Monday board based on DataFrame columns.
def new_columns_from_df(id, df):
    """
    Creates columns on a Monday board based on DataFrame columns.

    Args:
        board_id (int): The ID of the Monday board.
        df (DataFrame): The DataFrame containing column names.
    """
    columnsList = df.columns
    for i in columnsList :
        if all(isinstance(x, datetime) for x in df[i]) == True : type = "date"      # verifies if the column is a date type
        elif all(isinstance(x, int) for x in df[i]) == True : type = "numbers"        # verifies if the column is a int type
        elif all(isinstance(x, float) for x in df[i]) == True : type = "numbers"      # verifies if the column is a float type
        else : type = "text"                                                        # if it's not a date, will be a text column
        column = new_column(board_id=id, title=i, column_type=type)
        api = call_api(column)
        api.json

# Converts a DataFrame row to a dictionary.
def row_to_dict(row): 
    """
    Converts a DataFrame row to a dictionary.

    Args:
        row (Series): A row from a DataFrame.

    Returns:
        dict: A dictionary representation of the row.
    """
    rowDict = row.to_dict()
    return rowDict

# Converts a dictionary to JSON string.
def dict_to_json(elementDict):
    """
    Converts a dictionary to JSON string.

    Args:
        elementDict (dict): The dictionary to be converted.

    Returns:
        str: The JSON string representation of the dictionary.
    """
    rowJson = json.dumps(elementDict)
    return rowJson

# Finds a key in a dictionary by its value.
def find_key_by_value(dictionary, searchValue):
    """
    Finds a key in a dictionary by its value.

    Args:
        dictionary (dict): The dictionary to search.
        searchValue: The value to search for.

    Returns:
        Any: The key corresponding to the searchValue, or None if not found.
    """
    for key, value in dictionary.items():
        if value == searchValue:
            return key
    # If the value is not found, return None
    return None

# Replaces the original keys from a row by its Monday column IDs.
def match_keys_with_column_ids(rowDict:dict,idDict:dict): 
    """
    Replaces the original keys from a row by its Monday column IDs.

    Args:
        rowDict (dict): The dictionary containing row data.
        idDict (dict): The dictionary containing column IDs.

    Returns:
        dict: A new dictionary with keys replaced by column IDs.
    """
    newDict = {}
    for key in rowDict :
        newKey = find_key_by_value(idDict,key)
        if newKey :
            newDict[newKey] = rowDict[key]
    return newDict

# Converts Monday board JSON data to a DataFrame
def board_to_df(boardJsonList: list) :
    """
    Converts Monday board JSON data to a DataFrame.

    Args:
        boardJsonList (list): List of JSON data representing boards.

    Returns:
        DataFrame: The DataFrame containing board data.
    """
    boardTitles = boardJsonList[0]["data"]["boards"][0]["columns"]
    boardColumns = [j["title"] for j in boardTitles]
    boardTitles = boardJsonList[0]["data"]["boards"][0]["columns"]
    boardColumns = [j["title"] for j in boardTitles]
    boardDf = pd.DataFrame(columns=boardColumns)
    boardName = boardJsonList[0]["data"]["boards"][0]["name"]
    boardDf.insert(len(boardDf.columns) , "Mesa" , "")
    boardDf.insert(len(boardDf.columns) , "Grupo" , "")
    # boardDf.columns = map(str.upper , boardDf.columns)
    print(f'{boardName} successfully extracted')
    for boardJson in boardJsonList :
        if "boards" in boardJson["data"]:
            for i in boardJson["data"]["boards"][0]['items_page']['items'] :
                row = i["column_values"]
                row = list(map(lambda d : d["text"] , row))
                row.insert(0 , i["name"])
                row.insert(len(row) , boardName)
                row.insert(len(row) , i["group"]["title"])
                boardDf.loc[len(boardDf)] = row 
        elif "next_items_page" in boardJson["data"]:
            for i in boardJson['data']['next_items_page']['items'] :
                row = i["column_values"]
                row = list(map(lambda d : d["text"] , row))
                row.insert(0 , i["name"])
                row.insert(len(row) , boardName)
                row.insert(len(row) , i["group"]["title"])
                boardDf.loc[len(boardDf)] = row        
    return boardDf

# Creates the mandatory column 'Element' to be used as key on Monday
def create_element_column(dataframe, columns):
    """
    Creates a mandatory 'Element' column in the DataFrame.

    Args:
        dataframe (DataFrame): The DataFrame to which the column will be added.
        columns (list): List of columns to concatenate for generating 'Element' values.

    """
    dataframe["Element"] = dataframe[columns].apply(lambda row: ''.join(row.values.astype(str)), axis=1)

### CALL FUNCTION ###

In [ ]:
def call_api(query: str, url: str = "https://api.monday.com/v2", token: str = None) -> requests.Response:
    """
    Calls the Monday.com API.

    Args:
        query (str): GraphQL query string.
        url (str, optional): API URL. Defaults to "https://api.monday.com/v2".
        token (str, optional): API token. If not provided, it will be retrieved from the GitHub repository secret named "monday_token".

    Returns:
        Response: Response object from the API.
    """
    # If token is not provided, try to retrieve it from GitHub repository secret
    if token is None:
        token = os.environ.get("monday_token")

    if not token:
        raise ValueError("Monday.com API token not found. Please provide the token or set the 'monday_token' secret in your GitHub repository.")

    headers = {"Content-Type": "application/json", "Authorization": "Bearer " + token}
    payload = {"query": query}
    response = requests.post(url, json=payload, headers=headers)
    return response


### QUERY FUNCTIONS ###

In [ ]:
# Generates a GraphQL query to retrieve items from a Monday board
def get_items(board_id: int) -> str:
    """
    Generates a GraphQL query to retrieve items from a Monday board.

    Args:
        board_id (int): The ID of the Monday board.

    Returns:
        str: GraphQL query string.
    """
    query = """
        query {
            boards (ids: %s) {
                id
                name
                columns {
                    title
                }
                items_page (limit:500) {
                    cursor
                    items {
                        id
                        name
                        group {
                            title
                        }
                        created_at
                        column_values {
                            text
                        }
                    }
                }
            }
        }
    """ % (board_id)
    return query

# Generates a GraphQL query to retrieve the next items page of the board.
def next_items_page(cursor):
    """
    Generates a GraphQL query to retrieve the next items page of the board.
    This is only necessary if the board has more than 500 items

    Args:
        cursor (str): The cursor ID to continue the iteration through the board.

    Returns:
        str: GraphQL query string.
    """
    query = """
        query {
            next_items_page (limit:500, cursor:"%s") {
                cursor
                items {
                    id
                    name
                    group {
                        title
                    }
                    created_at
                    column_values {
                        text
                    }
                }
            }
        }
""" % (cursor)
    return query

# Generates a GraphQL query to retrieve item names from a Monday board
def get_item_names(board_id)  -> str:
  """
    Generates a GraphQL query to retrieve item names from a Monday board.

    Args:
        board_id (int): The ID of the Monday board.

    Returns:
        str: GraphQL query string.
    """
  query = """ 
  query {
    boards(ids: %s) {
      items_page {
        items {
          id
          name
        }
      }
    }
  }""" % (board_id)
  return query

# Generates a GraphQL query to retrieve the column IDs from a Monday board.
def get_column_ids(board_id) -> str:
    """
    Generates a GraphQL query to retrieve the column IDs from a Monday board.

    Args:
        board_id (int): The ID of the Monday board.

    Returns:
        str: GraphQL query string.
    """    
    query = """
    query {
        boards(ids: %s) {
            columns {
                id
              title
            }
        }
    }""" % (board_id)
    return query

# Retrieves a Monday board's data in JSON format.
def get_board(boardCode):
    """
    Retrieves a Monday board's data in JSON format.

    Args:
        boardCode (int): The ID of the Monday board to retrieve.

    Returns:
        list: A list containing JSON data representing the board.
    """
    boardJsonList = []
    print(f'Extracting board {boardCode}')
    results = get_items(boardCode)
    api = call_api(results)
    boardJson = api.json()
    boardJsonList.append(boardJson)
    cursor = boardJson['data']['boards'][0]['items_page']['cursor']
    lenCheck = len(boardJson['data']['boards'][0]['items_page']['items'])
    # Continue retrieving additional items while there are more pages
    while lenCheck == 500 :
        results = next_items_page(cursor)
        api = call_api(results)
        boardJson = api.json()
        boardJsonList.append(boardJson)
        cursor = boardJson['data']['next_items_page']['cursor']
        lenCheck = len(boardJson['data']['next_items_page']['items'])
    return boardJsonList


### MUTATION FUNCTIONS

In [ ]:
# Creates a new Monday board with the specified name and description
def new_public_board(board_name="My Board", description="") -> str : # if not defined, board name and description will be set to default
    """
    Generates a GraphQL mutation to create a new public Monday board.

    Args:
        board_name (str): The name of the new board (default is "My Board").
        description (str): The description of the new board (default is an empty string).

    Returns:
        str: GraphQL mutation string.
    """
    mutation =  """
    mutation {
        create_board(
          board_name: "%s",
          description: "%s",
          board_kind: public){id}
        }""" % (board_name, description)
    return  mutation

# Creates a new element in a Monday board with the specified item name and column values
def new_item(board_id,item_name,column_values) -> str:
  """
    Generates a GraphQL mutation to create a new item in a Monday board.

    Args:
        board_id (int): The ID of the target Monday board.
        item_name (str): The name of the new item.
        column_values (str): JSON string representing the column values of the new item.

    Returns:
        str: GraphQL mutation string.
  """
  mutation = """ 
  mutation {
    create_item (
      board_id: %s,
      item_name: "%s",
      column_values: %s) {
      id
    }
  }""" % (board_id,item_name,column_values)
  return mutation

# Deletes the specified item from a Monday board
def delete_item(item_id):
    """
    Generates a GraphQL mutation to delete an item from a Monday board.

    Args:
        item_id (int): The ID of the item to delete.

    Returns:
        str: GraphQL mutation string.
    """
    mutation = """
    mutation {
    delete_item (item_id: %s) {
        id
        }
    }
        """ % (item_id)
    return mutation

# Creates a single column in a Monday board with the specified title and column type
def new_column(board_id, title,column_type) -> str:
  """
    Generates a GraphQL mutation to create a new column in a Monday board.

    Args:
        board_id (int): The ID of the target Monday board.
        title (str): The title of the new column.
        column_type (str): The type of the new column (e.g., "text", "date", "numbers").

    Returns:
        str: GraphQL mutation string.
    """
  mutation = """
  mutation {
    create_column (board_id: %s,
                   title: "%s",
                   column_type: %s) {
                   id
    }
  }""" % (board_id, title,column_type)
  return mutation

# Creates each row from the assigned DataFrame as a Monday element in the specified board
def import_df_to_monday_board(board_id,dataframe) :
    """
    Imports each row from the assigned DataFrame as a Monday element in the specified board.

    Args:
        board_id (int): The ID of the target Monday board.
        dataframe (DataFrame): The DataFrame containing row data to be imported.

    Returns:
        None
    """
    
    # Converting datetime columns to string so they can be JSON serializable
    columnTypes = dataframe.dtypes
    for columnName, columnType in columnTypes.items() :
        if columnType == 'datetime64[ns]' :
            dataframe[columnName] = dataframe[columnName].astype(str)
    
    # Get column IDs from Monday and create a dictionary
    call = call_api(get_column_ids(board_id))
    call = call.json()
    idDict = {column['id']: column['title'] for column in call['data']['boards'][0]['columns']}

    # Iterate over each row in the DataFrame
    for index in range(len(dataframe)):
        rowDict = row_to_dict(dataframe.iloc[index])                            # Transforming dataframe row into a dictionary
        elementDict = match_keys_with_column_ids(rowDict,idDict)                # Updating dictionary keys with the column ids from their corresponding columns on Monday
        elementJsonString = dict_to_json(elementDict)
        elementJsonString = json.dumps(elementJsonString)                       # Transforming json object into a json string
        element = new_item(lastBoardId,rowDict['Element'],elementJsonString)  # Creating mutation query
        call_api(element)

## BOARD CREATION AND DATA MANIPULATION

In [ ]:
# Secrete token upload

In [ ]:
# Load the default sample database from the "Absenteeism_at_work_Project.xls" file.
sample = pd.read_excel("Absenteeism_at_work_Project.xls")

# Create a new public board named "Absenteeism_at_work" on Monday.com.
create = new_public_board(board_name="Absenteeism_at_work") 
api = call_api(create)
apiJson = api.json()
# Save the ID of the newly created board to a variable.
lastBoardId = apiJson['data']['create_board']['id']

# Print the ID of the newly created board.
print(f"The ID of your new board is: {lastBoardId}")

# Retrieve the first item from the newly created board.
firstItem = get_items(lastBoardId)
api = call_api(firstItem)
apiJson = api.json()
# Retrieve the ID of the first item on the board.
firstItemId = apiJson['data']['boards'][0]['items_page']['items'][0]['id']

# Delete the first item from the board.
deleteItem = delete_item(firstItemId)
api = call_api(deleteItem)
apiJson = api.json()


In [ ]:
# Create columns on the Monday board based on DataFrame columns
new_columns_from_df(lastBoardId, sample)

In [ ]:
# Create an identifier column for Monday elements based on the 'ID' column in the sample DataFrame
create_element_column(sample,['ID'])

In [ ]:
# Upload DataFrame rows to the last created Monday board
# Due to the structure of the Monday database, a separate query is executed for each row, which may lead to longer execution times for large datasets
import_df_to_monday_board(lastBoardId,sample)

In [ ]:
# Extract a Monday board's data as a list of JSON objects
boardJsonList = get_board(lastBoardId)

In [ ]:
# Transform a list of JSON objects into a single DataFrame
boardDf = board_to_df(boardJsonList)